In [1]:
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import requests

image = '/home/data2/zls/code/ABAW/emi/dataset/image/face_images/00000/0.jpg'
# image = 'ABAW/images/face_images/00000/0.jpg'
image = Image.open(image)

processor = AutoImageProcessor.from_pretrained('/home/data2/zls/code/ckpt/google/vit-base-patch16-224-in21k')
model = AutoModel.from_pretrained('/home/data2/zls/code/ckpt/google/vit-base-patch16-224-in21k')
# processor = AutoImageProcessor.from_pretrained('/data/maihn/Bert/vit-base-patch16-224-in21k')
# model = AutoModel.from_pretrained('/data/maihn/Bert/vit-base-patch16-224-in21k')

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state
last_hidden_states.shape

/home/zhuls24/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/data2/zls/envs/hugface/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/data2/zls/envs/hugface/lib/python3.10/site-packages/torchvision/

FileNotFoundError: [Errno 2] No such file or directory: '/home/data2/zls/code/ABAW/emi/dataset/image/face_images/00000/0.jpg'

In [ ]:
import soundfile as sf
import torch
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor.from_pretrained("/home/data2/zls/code/ckpt/facebook/wav2vec2-base-960h")
# processor = Wav2Vec2Processor.from_pretrained("/data/maihn/Bert/wav2vec2-base-960h")
print(processor)
audio_input, sample_rate = sf.read('/home/data2/zls/code/ABAW/emi/dataset/audio/00000.mp3')  # (31129,)
input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt")
# torch.Size([1, 31129])
print(input_values)
input_values = input_values.input_values
from transformers import Wav2Vec2Model
model = Wav2Vec2Model.from_pretrained("/home/data2/zls/code/ckpt/facebook/wav2vec2-base-960h")    # 用于提取通用特征，768维
wav2vec2 = model(input_values)['last_hidden_state']     # torch.Size([1, 97, 768])，模型出来是一个BaseModelOutput的结构体。
wav2vec2.shape

Wav2Vec2Processor:
- feature_extractor: Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

- tokenizer: Wav2Vec2CTCTokenizer(name_or_path='/home/data2/zls/code/ckpt/facebook/wav2vec2-base-960h', vocab_size=32, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'})
{'input_values': tensor([[ 0.0007,  0.0018, -0.0084,  ..., -0.0002, -0.0005, -0.0005]])}


Some weights of the model checkpoint at /home/data2/zls/code/ckpt/facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at /home/data2/zls/code/ckpt/facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


torch.Size([1, 294, 768])

In [ ]:
from pathlib import Path

image_dir = Path('/home/data2/zls/code/ABAW/emi/dataset/image/face_images')


: 

In [3]:
from pathlib import Path
from PIL import Image
import numpy as np
import os
from tqdm import tqdm

# 设置图片目录及输出目录
image_dir = Path('/home/data2/zls/code/ABAW/emi/dataset/test_data/images/face_images')
output_dir = Path('/home/data2/zls/code/ABAW/emi/dataset/test_data/images/averaged')
# image_dir = Path('/home/data2/zls/code/ABAW/emi/dataset/images/face_images')
# output_dir = Path('/home/data2/zls/code/ABAW/emi/dataset/images/averaged')
output_dir.mkdir(exist_ok=True)

# 定义目标尺寸（可根据ViT模型实际尺寸调整）
target_size = (224, 224)

# 遍历 image_dir 下的所有子文件夹
for folder in tqdm([d for d in image_dir.iterdir() if d.is_dir()]):
    image_files = list(folder.glob('*.[pjPJ]*[npNP]*'))  # 简单匹配 jpg/png 文件，可根据需要调整扩展名
    if not image_files:
        continue

    sum_array = None
    count = 0

    for img_path in image_files:
        try:
            img = Image.open(img_path).convert('RGB')
            img = img.resize(target_size, Image.Resampling.LANCZOS)
            img_array = np.array(img, dtype=np.float32)
            if sum_array is None:
                sum_array = img_array
            else:
                sum_array += img_array
            count += 1
        except Exception as e:
            print(f"无法处理 {img_path}: {e}")

    if count > 0:
        # 求平均
        avg_array = sum_array / count
        avg_img = Image.fromarray(np.uint8(avg_array))
        # 保存融合后的图片，以文件夹名命名保存结果
        output_path = output_dir / f"{folder.name}_average.png"
        avg_img.save(output_path)

100%|██████████| 4586/4586 [19:25<00:00,  3.93it/s]


In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
# from datasets import load_dataset
from pathlib import Path
from tqdm import tqdm
import soundfile as sf


device = "cuda:0" if torch.cuda.is_available() else "cpu"

model_id = "/home/data2/zls/code/ckpt/openai/whisper-large-v3-turbo"
# model_id = "/home/data2/zls/code/ckpt/whisper-large-v3"
# model_id = "openai/whisper-large-v3-turbo"
# model_id = "/home/data2/zls/code/ckpt/openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(model_id)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)


# audio_dir = Path('ABAW/audio')
audio_dir = Path('/home/data2/zls/code/ABAW/emi/dataset/test_data/audio')
audio_files = list(audio_dir.glob('*.mp3'))
# output_dir = Path('ABAW/text')
output_dir = Path('/home/data2/zls/code/ABAW/emi/dataset/test_data/text')

output_dir.mkdir(exist_ok=True)
output_dir_list = [f.stem for f in output_dir.glob("*.txt")]

for audio_file in tqdm(audio_files):
    if audio_file.stem in output_dir_list:
        continue
    try:
        audio_path = str(audio_file)
        audio, sample_rate = sf.read(audio_path)
        inputs = processor(audio, return_tensors="pt", sampling_rate=sample_rate).input_features
        inputs = inputs.to(device)
        with torch.no_grad():
            generated_ids = model.generate(input_features=inputs)
        result = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        output_file = output_dir / f"{audio_file.stem}.txt"
        with open(output_file, 'w') as f:
            f.write(result)
    except Exception as e:
        print(f"无法处理 {audio_file}: {e}")
            



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/zhuls24/.conda/envs/lavis/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/zhuls24/.conda/envs/lavis/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/zhuls24/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/zhuls24/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.sta

RuntimeError: Failed to import transformers.models.auto.modeling_auto because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject